In [ ]:
from astropy.time import Time, TimeDelta
import astropy.units as u
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', logo=False)

from lsst_efd_client import EfdClient

In [ ]:
start_date = Time("2024-04-18T12:00:00", scale="utc")
end_date= start_date + TimeDelta(val=3600*24 * u.s)
csc = "GenericCamera"
topics = ["command_takeImages", "logevent_startTakeImage", "logevent_endTakeImage", "command_startStreamingMode", "command_stopStreamingMode"]
efd_name = "usdf_efd"

In [ ]:
client = EfdClient(efd_name)

In [ ]:
data = []
for topic in topics:
    df = await client.select_time_series(f"lsst.sal.{csc}.{topic}",
                                         "salIndex",
                                         start_date,
                                         end_date)
    data.append(df)

In [ ]:
d0 = hv.Table(data[0].reset_index())
cti = hv.Points(d0, label="Command takeImages").opts(marker="o")

d1 = hv.Table(data[1].reset_index())
tis = hv.Points(d1, label="Start takeImage").opts(marker="+")

d2 = hv.Table(data[2].reset_index())
eti = hv.Points(d2, label="End takeImage").opts(marker="x")

d3 = hv.Table(data[3].reset_index())
if d3.rows > 1:
    sst = hv.Points(d3, label="Start streamingMode").opts(marker="^")
else:
    sst = None

d4 = hv.Table(data[4].reset_index())
if d4.rows > 1:
    est = hv.Points(d4, label="End streamingMode").opts(marker="v")
else:
    est = None

In [ ]:
layout = cti * tis * eti
if sst is not None and est is not None:
    layout *= sst * est
layout.opts(opts.Points(size=20, width=1000, height=600, legend_position="left", legend_offset=(0, 200),
                        xlabel="Time", yticks=[101, 102, 103]))

In [ ]:
eti_103 = d2[d2["salIndex"] == 103]
tis_103 = d1[d1["salIndex"] == 103]

In [ ]:
import numpy as np
diff_103 = np.abs(eti_103["index"][:167] - tis_103["index"][1:168])

In [ ]:
conv = diff_103 / np.timedelta64(1, "ns") / 1e9

In [ ]:
conv * 1000